In [9]:
import pandas as pd
import numpy as np
import copy 
import warnings
warnings.filterwarnings(action='ignore')

In [10]:
file = 'C:/Users/0630r/inha2/Research/Dart/Dart_file/support_data/dart-2020-2022.xlsx' # 20~22년도까지 원본 데이터
file2 = 'C:/Users/0630r/inha2/Research/Dart/Dart_file/support_data/dart-R&D19.xlsx' # R&D 19년도 4분기부터 있는 데이터

In [11]:
# 원본 데이터 finance 부분
df = pd.read_excel(file, sheet_name = [2], header = 3) # sheet 2번 읽기
df = df[2] # dict 타입으로 정렬되어있어 [2]로 지정 후 df 설정
df.to_parquet('orign_finance.pqt') # 원본 저장
df.head()

,symbol,entity_name,자산,유동자산,개발비,부채,매출,연구비,경상연구개발비,경상개발비,...,자산.11,유동자산.11,개발비.11,부채.11,매출.11,연구비.11,경상연구개발비.11,경상개발비.11,영업이익.11,총당기순이익.11
0,KRX:004310,현대약품,144423929000,71716241000,NaN,59073305000,3.213295e+10,NaN,3.242433e+09,NaN,...,181159428000,101642698000,NaN,9.460415e+10,1.626900e+11,NaN,8.375643e+09,NaN,7970051000,-164583000
1,KRX:000020,동화약품,376185904000,237916006000,NaN,76698758000,6.700367e+10,NaN,3.832272e+09,NaN,...,461136459000,227527471000,NaN,8.246321e+10,3.404263e+11,NaN,1.928457e+10,NaN,29915078000,21591211000
2,KRX:000040,KR모터스,157707823000,60080183000,3.264286e+09,121952871000,2.855757e+10,NaN,4.725640e+08,2.464500e+07,...,149185246000,60741213000,584196000.0,1.091612e+11,1.170587e+11,NaN,4.386520e+09,1.235000e+08,-6234393000,-13001280000
3,KRX:000050,경방,1406291293000,328282005000,NaN,673367459000,7.399416e+10,NaN,NaN,4.787000e+06,...,1248318314000,149882373000,NaN,4.933099e+11,3.892276e+11,NaN,NaN,2.419100e+07,31895306000,4415250000
4,KRX:000070,삼양홀딩스,3767622045000,1458808508000,NaN,1757928197000,5.973064e+11,NaN,NaN,6.807000e+09,...,4547780622000,1747899297000,NaN,2.024270e+12,3.316778e+12,NaN,NaN,2.562400e+10,132324277000,105849359000


In [12]:
# 19년도 4분기 개발비 항목만 추출 -> 이후 개발비 증가분 계산 
df2 = pd.read_excel(file2, sheet_name = [0], header = 3) # file2 읽기
df2 = df2[0] # dict 타입으로 정렬되어있어 [0]로 지정 후 df 설정
df2.rename(columns = {'개발비-01-2022-12-31':'개발비'}, inplace = True) # 개발비만 뽑기 위해 이름 변경
df2 = df2.iloc[:, :3] # symbol, entity_name, 개발비 순으로 데이터가 있어 3개의 열만 추출 
df2.insert(2, '연도', 2019) # 연도 설정
df2.insert(3, '분기', str(4)+'Q') # 분기 설정
df2.fillna(0,inplace = True) # 비어있는 부분은 0으로 채움
df2.to_parquet('orign_19R&D.pqt') # 19년도 개발비 원본 저장 
df2.head()

,symbol,entity_name,연도,분기,개발비
0,KRX:004310,현대약품,2019,4Q,3.679268e+09
1,KRX:000020,동화약품,2019,4Q,0.000000e+00
2,KRX:000040,KR모터스,2019,4Q,2.854131e+09
3,KRX:000050,경방,2019,4Q,0.000000e+00
4,KRX:000070,삼양홀딩스,2019,4Q,0.000000e+00


In [13]:
path = 'C:/Users/0630r/inha2/Research/Dart/Dart_file/real_code/pqt_file/'
df_fin = pd.read_parquet(path + 'orign_finance.pqt',engine = 'pyarrow') # 첫번째 df 불러오기
df_fin.head()

,symbol,entity_name,자산,유동자산,개발비,부채,매출,연구비,경상연구개발비,경상개발비,...,자산.11,유동자산.11,개발비.11,부채.11,매출.11,연구비.11,경상연구개발비.11,경상개발비.11,영업이익.11,총당기순이익.11
0,KRX:004310,현대약품,144423929000,71716241000,NaN,59073305000,3.213295e+10,NaN,3.242433e+09,NaN,...,181159428000,101642698000,NaN,9.460415e+10,1.626900e+11,NaN,8.375643e+09,NaN,7970051000,-164583000
1,KRX:000020,동화약품,376185904000,237916006000,NaN,76698758000,6.700367e+10,NaN,3.832272e+09,NaN,...,461136459000,227527471000,NaN,8.246321e+10,3.404263e+11,NaN,1.928457e+10,NaN,29915078000,21591211000
2,KRX:000040,KR모터스,157707823000,60080183000,3.264286e+09,121952871000,2.855757e+10,NaN,4.725640e+08,2.464500e+07,...,149185246000,60741213000,584196000.0,1.091612e+11,1.170587e+11,NaN,4.386520e+09,1.235000e+08,-6234393000,-13001280000
3,KRX:000050,경방,1406291293000,328282005000,NaN,673367459000,7.399416e+10,NaN,NaN,4.787000e+06,...,1248318314000,149882373000,NaN,4.933099e+11,3.892276e+11,NaN,NaN,2.419100e+07,31895306000,4415250000
4,KRX:000070,삼양홀딩스,3767622045000,1458808508000,NaN,1757928197000,5.973064e+11,NaN,NaN,6.807000e+09,...,4547780622000,1747899297000,NaN,2.024270e+12,3.316778e+12,NaN,NaN,2.562400e+10,132324277000,105849359000


In [14]:
## 내가 진행한 방법: symbol, entity_name은 고정이고 자산~총당기순이익이 반복되기 때문에 총당기순이익 부분을 중심으로 인덱스를 찾고 데이터를 분기별로 짤라서 axis = 1로 concat을 진행한다. 
## 이후 이름별과 분기별로 정렬 

In [15]:
## 총당기순이익이 있는 열의 위치를 파악
column_posit = [pos for pos,col in enumerate(df_fin.columns) if '총당기순이익' in col] # for문 먼저 진행 이후 if문 진행
print(column_posit)

[11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121]


In [16]:
## 이거 연도, 분기 만든거 없애서 코드 수정 -> 본 데이터는 연도분기 합치니깐 

In [17]:
## 2020 finance 정리
df_fin20 = df_fin.iloc[:,:column_posit[0]] # 첫번째 당기순이익이 있는 열까지 1분기로 설정
df_fin20.insert(2, '연도', 2020) # 연도 추가
df_fin20.insert(3, '분기', '1Q') # 분기 추가
n20 = 2
for i in range(1,4): # 2~4분기로 모두 구분이후 나중에 concat
    df_fin2 = df_fin.iloc[:,:column_posit[i]+1] # 22,32,42 (1분기 자산 위치)
    df_fin2.drop(df_fin.iloc[:, 2:column_posit[i-1]+1], axis = 1, inplace = True) # 1:[2:12], 2:[2:22], 3:[2:32] drop
    df_fin2.insert(2, '연도', 2020) # 연도 추가
    df_fin2.insert(3, '분기', str(n20)+'Q') # 분기 넣기
    df_fin2.columns = df_fin2.columns.str.replace('.'+str(n20-1),'') # 뒤에 .1, .2 붙은 것들 전부 삭제 
    n20 += 1
    df_fin20 = pd.concat([df_fin20, df_fin2], axis = 0, ignore_index = True)

## 2021 finance 정리
n21 = 1
df_fin21 = pd.DataFrame()
for i in range(4,8): # 이후 반복
    df_fin2 = df_fin.iloc[:,:column_posit[i]+1] # 22,32,42 
    df_fin2.drop(df_fin2.iloc[:, 2:column_posit[i-1]+1], axis = 1, inplace = True) # [0,1,2]
    df_fin2.insert(2, '연도', 2021)
    df_fin2.insert(3, '분기', str(n21)+'Q')
    df_fin2.columns = df_fin2.columns.str.replace('.'+str(n21+3),'')
    n21 += 1
    df_fin21 = pd.concat([df_fin21, df_fin2], axis = 0, ignore_index = True)

## 2022 finance 정리
n22 = 1
df_fin22 = pd.DataFrame()
for i in range(8,12): # 1,2,3
    df_fin2 = df_fin.iloc[:,:column_posit[i]+1] # 22,32,42 
    df_fin2.drop(df_fin2.iloc[:, 2:column_posit[i-1]+1], axis = 1, inplace = True) # [0,1,2]
    df_fin2.insert(2, '연도', 2022)
    df_fin2.insert(3, '분기', str(n22)+'Q')
    df_fin2.columns = df_fin2.columns.str.replace('.'+str(n22+7),'')
    n22 += 1
    df_fin22 = pd.concat([df_fin22, df_fin2], axis = 0, ignore_index = True)

In [52]:
fin_tt = pd.concat([df_fin20, df_fin21, df_fin22]) # 3개년 모두 concat
fin_tt.reset_index(drop=True, inplace = True) 
fin_tt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30192 entries, 0 to 30191
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   symbol       30192 non-null  object 
 1   entity_name  30192 non-null  object 
 2   연도           30192 non-null  int64  
 3   분기           30192 non-null  object 
 4   자산           30192 non-null  int64  
 5   유동자산         30192 non-null  int64  
 6   개발비          9068 non-null   float64
 7   부채           30189 non-null  float64
 8   매출           29752 non-null  float64
 9   연구비          4876 non-null   float64
 10  경상연구개발비      15386 non-null  float64
 11  경상개발비        8330 non-null   float64
 12  영업이익         30188 non-null  float64
 13  총당기순이익       27673 non-null  float64
dtypes: float64(8), int64(3), object(3)
memory usage: 3.2+ MB


### 개발비 증가분 계산 진행

In [53]:
fin_tt['개발비'].fillna(0, inplace = True) # 데이터를 신뢰하여 null값은 모두 0으로 대체 
fin_tt2 = fin_tt.loc[:,['symbol', 'entity_name','연도','분기','개발비']]
fin_tt2.head()

,symbol,entity_name,연도,분기,개발비
0,KRX:004310,현대약품,2020,1Q,0.000000e+00
1,KRX:000020,동화약품,2020,1Q,0.000000e+00
2,KRX:000040,KR모터스,2020,1Q,3.264286e+09
3,KRX:000050,경방,2020,1Q,0.000000e+00
4,KRX:000070,삼양홀딩스,2020,1Q,0.000000e+00


In [54]:
df2 = pd.read_parquet(path + 'orign_19R&D.pqt', engine = 'pyarrow')
fin_t = pd.concat([df2, fin_tt2], axis =0) # 본 데이터와 19년도 데이터 병합
fin_t1 = fin_t.sort_values(by = ['symbol', 'entity_name']).reset_index(drop=True)
fin_t1.head()

,symbol,entity_name,연도,분기,개발비
0,KRX:000020,동화약품,2019,4Q,0.0
1,KRX:000020,동화약품,2020,1Q,0.0
2,KRX:000020,동화약품,2020,2Q,0.0
3,KRX:000020,동화약품,2020,3Q,0.0
4,KRX:000020,동화약품,2020,4Q,0.0


In [55]:
fin_t1['개발비증가분'] = fin_t1['개발비'].diff() # 전 분기 대비 개발비 증가분 계산
fin_t1 =fin_t1.drop(columns = '개발비', axis =1) # 개발비 열 삭제
fin_t1.drop(fin_t1[fin_t1['연도'] == 2019].index, axis = 0, inplace = True) # 19년도행 제거
fin_t1.drop(fin_t1[fin_t1.duplicated(['symbol', '연도', '분기'])].index, axis = 0, inplace = True) # 중복행 삭제 
fin_t1.reset_index(drop=True, inplace = True) 
fin_t1

,symbol,entity_name,연도,분기,개발비증가분
0,KRX:000020,동화약품,2020,1Q,0.0
1,KRX:000020,동화약품,2020,2Q,0.0
2,KRX:000020,동화약품,2020,3Q,0.0
3,KRX:000020,동화약품,2020,4Q,0.0
4,KRX:000020,동화약품,2021,1Q,0.0
...,...,...,...,...,...
30187,KRX:950220,네오이뮨텍,2021,4Q,0.0
30188,KRX:950220,네오이뮨텍,2022,1Q,0.0
30189,KRX:950220,네오이뮨텍,2022,2Q,0.0
30190,KRX:950220,네오이뮨텍,2022,3Q,0.0


In [56]:
fin_t1.개발비증가분 = np.where(fin_t1['개발비증가분'] <= 0, 0, fin_t1.개발비증가분) # 0으로 지정
fin_t1.개발비증가분.describe()
## 개발비 증가분의 경우 전년도에 비해 (-)가 나오는 비용이 존재함 -> 즉 1. (개발비>0) < (상각>0) 2. (개발비=0), (상각비>0)  -> 그래서 이 경우 최종적을 R&D 비용을 구하는 거기 때문에 1의 경우는 알 수 없으니 (-)이면 2번으로 판단하여 개발비가 없다고 가정함

count    3.019200e+04
mean     1.450509e+09
std      5.714141e+10
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      4.441848e+12
Name: 개발비증가분, dtype: float64

In [57]:
fin_t1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30192 entries, 0 to 30191
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   symbol       30192 non-null  object 
 1   entity_name  30192 non-null  object 
 2   연도           30192 non-null  int64  
 3   분기           30192 non-null  object 
 4   개발비증가분       30192 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ MB


### 원본 데이터와 개발비 증가분 구한 데이터 병합

In [58]:
col = ['symbol', 'entity_name', '연도', '분기']
fin_tt = fin_tt.merge(fin_t1, on = col) # 데이터 병합
fin_tt.head()

,symbol,entity_name,연도,분기,자산,유동자산,개발비,부채,매출,연구비,경상연구개발비,경상개발비,영업이익,총당기순이익,개발비증가분
0,KRX:004310,현대약품,2020,1Q,144423929000,71716241000,0.000000e+00,5.907330e+10,3.213295e+10,NaN,3.242433e+09,NaN,2.783020e+08,NaN,0.0
1,KRX:000020,동화약품,2020,1Q,376185904000,237916006000,0.000000e+00,7.669876e+10,6.700367e+10,NaN,3.832272e+09,NaN,2.706892e+09,NaN,0.0
2,KRX:000040,KR모터스,2020,1Q,157707823000,60080183000,3.264286e+09,1.219529e+11,2.855757e+10,NaN,4.725640e+08,2.464500e+07,3.859850e+08,NaN,410155000.0
3,KRX:000050,경방,2020,1Q,1406291293000,328282005000,0.000000e+00,6.733675e+11,7.399416e+10,NaN,NaN,4.787000e+06,3.175385e+09,NaN,0.0
4,KRX:000070,삼양홀딩스,2020,1Q,3767622045000,1458808508000,0.000000e+00,1.757928e+12,5.973064e+11,NaN,NaN,6.807000e+09,3.258234e+10,NaN,0.0


In [59]:
fin_tt.drop(fin_tt[fin_tt['매출'].isnull()].index, axis = 0, inplace = True) # 매출이 null인 기업들 삭제 
fin_tt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29752 entries, 0 to 30182
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   symbol       29752 non-null  object 
 1   entity_name  29752 non-null  object 
 2   연도           29752 non-null  int64  
 3   분기           29752 non-null  object 
 4   자산           29752 non-null  int64  
 5   유동자산         29752 non-null  int64  
 6   개발비          29752 non-null  float64
 7   부채           29752 non-null  float64
 8   매출           29752 non-null  float64
 9   연구비          4876 non-null   float64
 10  경상연구개발비      15303 non-null  float64
 11  경상개발비        8317 non-null   float64
 12  영업이익         29752 non-null  float64
 13  총당기순이익       27255 non-null  float64
 14  개발비증가분       29752 non-null  float64
dtypes: float64(9), int64(3), object(3)
memory usage: 3.6+ MB


In [60]:
fin_tt[fin_tt['총당기순이익'].isnull()]['entity_name'].head(30) # 두산 기아, sk하이닉스 등을 버려야 하는 대기업들이 많아 회귀분석시에 당기순이익을 사용하지 않음으로 fillna(0) 진행

0         현대약품
1         동화약품
2        KR모터스
3           경방
4        삼양홀딩스
5        하이트진로
6         유한양행
7       CJ대한통운
8     하이트진로홀딩스
9           두산
10      성창기업지주
11          DL
12        유유제약
13       일동홀딩스
14      한국앤컴퍼니
15       삼천당제약
16          기아
17       대유플러스
18       노루홀딩스
19       삼화페인트
20        대원강업
21      중앙에너비스
22        조선내화
23          대동
24        가온전선
25        삼일제약
26       CS홀딩스
27    동아쏘시오홀딩스
28        천일고속
29      SK하이닉스
Name: entity_name, dtype: object

In [61]:
fin_tt.fillna(0, inplace = True) # 위에서 나머지 null값이 나타낸 column들은 TobinsQ에 영향을 끼치지 않고 충분히 0원이 가능할 수 있다고 판단하여 0으로 null값 대체 

In [62]:
fin_tt[fin_tt['경상연구개발비'] < 0] # 0보다 작은 애들 3개 존재

,symbol,entity_name,연도,분기,자산,유동자산,개발비,부채,매출,연구비,경상연구개발비,경상개발비,영업이익,총당기순이익,개발비증가분
943,KRX:053700,삼보모터스,2020,1Q,751268944000,332808057000,0.0,5.408537e+11,2.126944e+11,0.0,-224700000.0,0.0,1.627240e+08,0.000000e+00,0.0
6215,KRX:079650,서산,2020,3Q,112683828000,71139189000,0.0,1.230375e+10,2.625399e+10,0.0,-4184000.0,0.0,-7.139335e+09,-1.096227e+10,0.0
16519,KRX:104540,코렌텍,2021,3Q,97633473000,62259131000,0.0,4.561291e+10,3.138668e+10,0.0,-409361000.0,0.0,2.763731e+09,2.058827e+09,0.0


In [63]:
fin_tt['경상연구개발비'] = np.where(fin_tt['경상연구개발비'] < 0, 0, fin_tt['경상연구개발비']) # 특이값 0으로 처리

In [64]:
fin_tt['연구개발비'] = fin_tt['개발비증가분'] + fin_tt['연구비'] + fin_tt['경상연구개발비'] + fin_tt['경상개발비'] # R&D 비용 정리 
fin_tt.drop(columns = ['개발비', '연구비', '경상연구개발비', '경상개발비', '개발비증가분'], axis= 1, inplace = True) 
fin_tt

,symbol,entity_name,연도,분기,자산,유동자산,부채,매출,영업이익,총당기순이익,연구개발비
0,KRX:004310,현대약품,2020,1Q,144423929000,71716241000,5.907330e+10,3.213295e+10,2.783020e+08,0.000000e+00,3.242433e+09
1,KRX:000020,동화약품,2020,1Q,376185904000,237916006000,7.669876e+10,6.700367e+10,2.706892e+09,0.000000e+00,3.832272e+09
2,KRX:000040,KR모터스,2020,1Q,157707823000,60080183000,1.219529e+11,2.855757e+10,3.859850e+08,0.000000e+00,9.073640e+08
3,KRX:000050,경방,2020,1Q,1406291293000,328282005000,6.733675e+11,7.399416e+10,3.175385e+09,0.000000e+00,4.787000e+06
4,KRX:000070,삼양홀딩스,2020,1Q,3767622045000,1458808508000,1.757928e+12,5.973064e+11,3.258234e+10,0.000000e+00,6.807000e+09
...,...,...,...,...,...,...,...,...,...,...,...
30131,KRX:448780,마이크로엔엑스,2022,4Q,23978351000,9314971000,1.452456e+10,8.816271e+09,1.060784e+09,7.525130e+08,6.012260e+08
30132,KRX:456190,큐라켐,2022,4Q,13953759000,6010287000,6.247284e+09,6.019995e+09,1.077072e+09,1.077692e+09,1.010160e+08
30154,KRX:432320,KB스타리츠,2022,4Q,1038998705000,100719848000,5.333309e+11,2.387691e+10,-4.405355e+09,1.595820e+10,0.000000e+00
30170,KRX:441270,파인엠텍,2022,4Q,272612584000,106129054000,1.300483e+11,1.163728e+11,5.442629e+09,-7.039643e+09,1.852383e+09


In [65]:
fin_tt['연구개발비'].describe()

count    2.975200e+04
mean     1.355396e+10
std      3.284489e+11
min      0.000000e+00
25%      0.000000e+00
50%      2.499250e+08
75%      1.796223e+09
max      2.500422e+13
Name: 연구개발비, dtype: float64

In [66]:
fin_tt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29752 entries, 0 to 30182
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   symbol       29752 non-null  object 
 1   entity_name  29752 non-null  object 
 2   연도           29752 non-null  int64  
 3   분기           29752 non-null  object 
 4   자산           29752 non-null  int64  
 5   유동자산         29752 non-null  int64  
 6   부채           29752 non-null  float64
 7   매출           29752 non-null  float64
 8   영업이익         29752 non-null  float64
 9   총당기순이익       29752 non-null  float64
 10  연구개발비        29752 non-null  float64
dtypes: float64(5), int64(3), object(3)
memory usage: 2.7+ MB


In [67]:
fin_tt.to_parquet('finance_final.pqt')
# 완

In [7]:
df = pd.read_parquet(path + 'finance_final.pqt', engine = 'pyarrow')
df

,symbol,entity_name,연도,분기,자산,유동자산,부채,매출,영업이익,총당기순이익,연구개발비
0,KRX:004310,현대약품,2020,1Q,144423929000,71716241000,5.907330e+10,3.213295e+10,2.783020e+08,NaN,NaN
1,KRX:000020,동화약품,2020,1Q,376185904000,237916006000,7.669876e+10,6.700367e+10,2.706892e+09,NaN,NaN
2,KRX:000040,KR모터스,2020,1Q,157707823000,60080183000,1.219529e+11,2.855757e+10,3.859850e+08,NaN,NaN
3,KRX:000050,경방,2020,1Q,1406291293000,328282005000,6.733675e+11,7.399416e+10,3.175385e+09,NaN,NaN
4,KRX:000070,삼양홀딩스,2020,1Q,3767622045000,1458808508000,1.757928e+12,5.973064e+11,3.258234e+10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
30187,KRX:448740,삼성스팩8호,2022,4Q,7979967000,7979967000,3.758510e+09,NaN,-7.118000e+06,-29780000.0,NaN
30188,KRX:448760,IBKS제22호스팩,2022,4Q,1976956000,1976956000,1.224360e+09,NaN,-1.846000e+07,-26150000.0,NaN
30189,KRX:448830,미래에셋비전스팩3호,2022,4Q,2991408000,2991408000,2.078731e+09,NaN,-8.069000e+06,-19579000.0,NaN
30190,KRX:449020,유안타제13호스팩,2022,4Q,2992616000,2992616000,1.407900e+09,NaN,-1.045200e+07,-8124000.0,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30192 entries, 0 to 30191
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   symbol       30192 non-null  object 
 1   entity_name  30192 non-null  object 
 2   연도           30192 non-null  int64  
 3   분기           30192 non-null  object 
 4   자산           30192 non-null  int64  
 5   유동자산         30192 non-null  int64  
 6   부채           30189 non-null  float64
 7   매출           29752 non-null  float64
 8   영업이익         30188 non-null  float64
 9   총당기순이익       27673 non-null  float64
 10  연구개발비        3048 non-null   float64
dtypes: float64(5), int64(3), object(3)
memory usage: 2.8+ MB


In [84]:
## orign_finance, orign_19R&D, finance_final 3개 저장